In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Datacamp Pyspark Tutorial").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/15 11:49:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('SalesAnalysis.csv',header=True,escape="\"")

In [4]:
df.show(5,0)

+--------+--------------------------+----------------+----------+--------------+------------------------------------+
|Order ID|Product                   |Quantity Ordered|Price Each|Order Date    |Purchase Address                    |
+--------+--------------------------+----------------+----------+--------------+------------------------------------+
|176558  |USB-C Charging Cable      |2               |11.95     |04/19/19 08:46|917 1st St, Dallas, TX 75001        |
|NULL    |NULL                      |NULL            |NULL      |NULL          |NULL                                |
|176559  |Bose SoundSport Headphones|1               |99.99     |04/07/19 22:30|682 Chestnut St, Boston, MA 02215   |
|176560  |Google Phone              |1               |600       |04/12/19 14:38|669 Spruce St, Los Angeles, CA 90001|
|176560  |Wired Headphones          |1               |11.99     |04/12/19 14:38|669 Spruce St, Los Angeles, CA 90001|
+--------+--------------------------+----------------+--

In [5]:
df.count()

186850

In [12]:
df.select('Product').distinct().count()

In [48]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df.groupBy('Product').agg(count('Quantity Ordered').alias('product_sale')).show()

+--------------------+------------+
|             Product|product_sale|
+--------------------+------------+
|    Wired Headphones|       18882|
|  Macbook Pro Laptop|        4724|
|Apple Airpods Hea...|       15549|
|              iPhone|        6842|
|                NULL|           0|
|Lightning Chargin...|       21658|
|Bose SoundSport H...|       13325|
|USB-C Charging Cable|       21903|
|AAA Batteries (4-...|       20641|
|        20in Monitor|        4101|
|    27in FHD Monitor|        7507|
|     Vareebadd Phone|        2065|
|34in Ultrawide Mo...|        6181|
|            LG Dryer|         646|
|AA Batteries (4-p...|       20577|
|        Google Phone|        5525|
|       Flatscreen TV|        4800|
|  LG Washing Machine|         666|
|             Product|         355|
|27in 4K Gaming Mo...|        6230|
+--------------------+------------+
only showing top 20 rows



In [22]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df = df.withColumn('Order Date',to_timestamp("Order Date", 'MM/dd/yy HH:mm'))
df.select(max("Order Date")).show()

+-------------------+
|    max(Order Date)|
+-------------------+
|2020-01-01 05:13:00|
+-------------------+



In [24]:
df.select(min("Order Date")).show()

+-------------------+
|    min(Order Date)|
+-------------------+
|2019-01-01 03:07:00|
+-------------------+



In [25]:
df.show(5,0)

+--------+--------------------------+----------------+----------+-------------------+------------------------------------+
|Order ID|Product                   |Quantity Ordered|Price Each|Order Date         |Purchase Address                    |
+--------+--------------------------+----------------+----------+-------------------+------------------------------------+
|176558  |USB-C Charging Cable      |2               |11.95     |2019-04-19 08:46:00|917 1st St, Dallas, TX 75001        |
|NULL    |NULL                      |NULL            |NULL      |NULL               |NULL                                |
|176559  |Bose SoundSport Headphones|1               |99.99     |2019-04-07 22:30:00|682 Chestnut St, Boston, MA 02215   |
|176560  |Google Phone              |1               |600       |2019-04-12 14:38:00|669 Spruce St, Los Angeles, CA 90001|
|176560  |Wired Headphones          |1               |11.99     |2019-04-12 14:38:00|669 Spruce St, Los Angeles, CA 90001|
+--------+------

In [26]:
df = df.withColumn("from_date", lit("01/01/2019 03:07"))
df = df.withColumn('from_date',to_timestamp("from_date", 'MM/dd/yy HH:mm'))

df2=df.withColumn('from_date',to_timestamp(col('from_date'))).withColumn('recency',col("Order Date").cast("long") - col('from_date').cast("long"))

In [27]:
df2 = df2.join(df2.groupBy('Order ID').agg(max('recency').alias('recency')),on='recency',how='leftsemi')

In [28]:
df2.show(5,0)

+--------+--------+--------------------------+----------------+----------+-------------------+------------------------------------+-------------------+
|recency |Order ID|Product                   |Quantity Ordered|Price Each|Order Date         |Purchase Address                    |from_date          |
+--------+--------+--------------------------+----------------+----------+-------------------+------------------------------------+-------------------+
|9351540 |176558  |USB-C Charging Cable      |2               |11.95     |2019-04-19 08:46:00|917 1st St, Dallas, TX 75001        |2019-01-01 03:07:00|
|8364180 |176559  |Bose SoundSport Headphones|1               |99.99     |2019-04-07 22:30:00|682 Chestnut St, Boston, MA 02215   |2019-01-01 03:07:00|
|8767860 |176560  |Google Phone              |1               |600       |2019-04-12 14:38:00|669 Spruce St, Los Angeles, CA 90001|2019-01-01 03:07:00|
|8767860 |176560  |Wired Headphones          |1               |11.99     |2019-04-12 14:

In [29]:
df2.printSchema()

root
 |-- recency: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: timestamp (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- from_date: timestamp (nullable = true)



In [39]:
df_freq = df2.groupBy('Product').agg(count('Quantity Ordered').alias('frequency'))

In [40]:
df_freq.show(5,0)

+------------------------+---------+
|Product                 |frequency|
+------------------------+---------+
|Wired Headphones        |18882    |
|Macbook Pro Laptop      |4724     |
|Apple Airpods Headphones|15549    |
|iPhone                  |6842     |
|Lightning Charging Cable|21658    |
+------------------------+---------+
only showing top 5 rows

